In [14]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold
#warnings.filterwarnings('ignore')
#%matplotlib inline
from sklearn.metrics import roc_auc_score
## 数据降维处理的
from sklearn.model_selection import train_test_split  
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

In [15]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
sub=pd.read_csv("submission.csv")
train

,policy_id,age,customer_months,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud
0,122576,37,189,2013-08-21,C,500/1000,1000,1465.71,5000000,455456,...,3,?,54930,6029,5752,44452,Nissan,Maxima,2000,0
1,937713,44,234,1998-01-04,B,250/500,500,821.24,0,591805,...,1,YES,50680,5376,10156,37347,Honda,Civic,1996,0
2,680237,33,23,1996-02-06,B,500/1000,1000,1844.00,0,442490,...,1,NO,47829,4460,9247,33644,Jeep,Wrangler,2002,0
3,513080,42,210,2008-11-14,A,500/1000,500,1867.29,0,439408,...,2,YES,68862,11043,5955,53548,Suburu,Legacy,2003,1
4,192875,29,81,2002-01-08,A,100/300,1000,816.25,0,640575,...,1,YES,59726,5617,10301,41550,Ford,F150,2004,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1008425,37,196,1997-06-29,C,250/500,500,1301.20,0,474615,...,3,NO,61433,10436,11432,39745,Nissan,Pathfinder,2011,1
696,770702,43,229,2001-05-29,A,250/500,500,1434.94,8000000,444476,...,1,?,68623,6798,14557,50606,Volkswagen,Passat,2013,1
697,755099,35,209,2003-01-11,C,100/300,500,1639.46,0,639608,...,0,YES,58033,9129,4598,40740,Mercedes,C300,2002,0
698,693804,44,275,2003-07-22,B,500/1000,2000,1042.29,0,432061,...,0,NO,35253,7359,3464,24677,Audi,A3,2007,1


In [16]:
data=pd.concat([train,test])
data['incident_date']

0      2014-12-22
1      2015-02-18
2      2015-01-18
3      2015-02-02
4      2015-02-09
          ...    
295    2015-01-14
296    2015-02-09
297    2014-12-21
298    2015-01-27
299    2015-01-29
Name: incident_date, Length: 1000, dtype: object

In [17]:
data['incident_date'] = pd.to_datetime(data['incident_date'],format='%Y-%m-%d')
data['incident_date']

0     2014-12-22
1     2015-02-18
2     2015-01-18
3     2015-02-02
4     2015-02-09
         ...    
295   2015-01-14
296   2015-02-09
297   2014-12-21
298   2015-01-27
299   2015-01-29
Name: incident_date, Length: 1000, dtype: datetime64[ns]

In [18]:
startdate = datetime.datetime.strptime('2022-06-30', '%Y-%m-%d')
startdate

datetime.datetime(2022, 6, 30, 0, 0)

In [19]:
data['time'] = data['incident_date'].apply(lambda x: startdate-x).dt.days
data['time']

0      2747
1      2689
2      2720
3      2705
4      2698
       ... 
295    2724
296    2698
297    2748
298    2711
299    2709
Name: time, Length: 1000, dtype: int64

In [20]:
#Encoder
numerical_fea = list(data.select_dtypes(include=['object']).columns)
division_le = LabelEncoder()
for fea in numerical_fea:
    division_le.fit(data[fea].values)
    data[fea] = division_le.transform(data[fea].values) 
print("数据预处理完成!")


数据预处理完成!


In [ ]:
testA=data[data['fraud'].isnull()].drop(['policy_id','incident_date','fraud'],axis=1)
trainA=data[data['fraud'].notnull()]
data_x=trainA.drop(['policy_id','incident_date','fraud'],axis=1)
data_y=train[['fraud']].copy()
col=['policy_state','insured_sex','insured_education_level','incident_type','collision_type','incident_severity','authorities_contacted','incident_state',
     'incident_city','police_report_available','auto_make','auto_model']
for i in data_x.columns:
    if i in col:
        data_x[i] = data_x[i].astype('str')
for i in testA.columns:
    if i in col:
        testA[i] = testA[i].astype('str')     

In [22]:
  
model=CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="AUC",
            task_type="GPU",
            learning_rate=0.1,
            iterations=10000,
            random_seed=2020,
            od_type="Iter",
            depth=7,
            early_stopping_rounds=300)
            
answers = []
mean_score = 0
n_folds = 10
sk = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2019)
for train, test in sk.split(data_x, data_y):
    x_train = data_x.iloc[train]
    y_train = data_y.iloc[train]
    x_test = data_x.iloc[test]
    y_test = data_y.iloc[test]
    clf = model.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=500,cat_features=col)
    yy_pred_valid=clf.predict(x_test)
    print('cat验证的auc:{}'.format(roc_auc_score(y_test, yy_pred_valid)))
    mean_score += roc_auc_score(y_test, yy_pred_valid) / n_folds
    y_pred_valid = clf.predict(testA,prediction_type='Probability')[:,-1]
    answers.append(y_pred_valid)
print('10折平均Auc:{}'.format(mean_score))
lgb_pre=sum(answers)/n_folds
sub['fraud']=lgb_pre
sub.to_csv('金融2预测.csv',index=False) 

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7612180	best: 0.7612180 (0)	total: 106ms	remaining: 17m 44s
bestTest = 0.8269231021
bestIteration = 3
Shrink model to first 4 iterations.
cat验证的auc:0.766025641025641


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7179487	best: 0.7179487 (0)	total: 67.2ms	remaining: 11m 12s
bestTest = 0.8653846383
bestIteration = 24
Shrink model to first 25 iterations.
cat验证的auc:0.8760683760683761


Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.7179487	best: 0.7179487 (0)	total: 66.1ms	remaining: 11m 1s
bestTest = 0.8338675201
bestIteration = 7
Shrink model to first 8 iterations.
cat验证的auc:0.7104700854700855


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7313034	best: 0.7313034 (0)	total: 71.5ms	remaining: 11m 54s
bestTest = 0.855769217
bestIteration = 4
Shrink model to first 5 iterations.
cat验证的auc:0.7745726495726496


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7177503	best: 0.7177503 (0)	total: 63.9ms	remaining: 10m 38s
500:	test: 0.8926728	best: 0.8957688 (376)	total: 34.6s	remaining: 10m 55s
bestTest = 0.8957688212
bestIteration = 376
Shrink model to first 377 iterations.
cat验证的auc:0.7435500515995872
10折平均Auc:0.7942268000317537
